## 1. Using just triplets

In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.globals import set_debug, set_verbose, set_llm_cache
from langchain_community.cache import InMemoryCache

set_debug(False)
set_verbose(False)
set_llm_cache(InMemoryCache())

In [2]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

model = ChatOpenAI(
    model="gpt-4o",
    api_key=openai_api_key
)

In [3]:
prompt_str = (
    f"You are an expert in understanding family relationships."
    f"You are given a list of relationships between individuals in the form of (Person A, Relationship, Person B)"
    f"Your task is to figure out the relationship :- {{question}}"
    f""
    f"list of relationships :"
    f"{{relationships}} are given a list of relationships as tripples"
)

prompt = ChatPromptTemplate.from_template(prompt_str)

In [4]:
inputs = [
    {
        "relationships": (
            "(Bernardo, has father, Antonio),"
            "(Antonio, has son, Bernardo),"
            "(Antonio, has father, Harold),"
            "(Harold, has son, Antonio),"
            "(Antonio, has mother, Shantel),"
            "(Shantel, has son, Antonio),"
            "(Shantel, has daughter, Marie),"
            "(Marie, has mother, Shantel)"
        ),
        "question": "Harold is Marie's ____"
    },
    {
        "relationships": (
            "(Robert, has brother, Antonio),"
            "(Robert, has daughter, Michelle),"
            "(Marie, has daughter, Michelle),"
            "(Marie, has baby, Baby),"
            "(Marie, has mother, Shantel),"
            "(Shantel, has granddaughter, Baby)"
        ),
        "question": "Shantel is Antonio's ____"
    },
    {
        "relationships": (
            "(Samuel, has grandfather, Harold),"
            "(Shantel, has son, Louis),"
            "(Harold, has daughter, Tracy),"
            "(Samuel, has son, Louis)"
        ),
        "question": "Tracy is Shantel's ____"
    },
    {
        "relationships": (
            "(Angela, has daughter, Nancy),"
            "(Hattie, has father, Samuel),"
            "(Samuel, has brother, Milton),"
            "(Hattie, has sister, Nancy),"
            "(Milton, has daughter, Arlene)"
        ),
        "question": "Arlene is Angela's ____"
    }
]

In [5]:
chain = prompt | model | StrOutputParser()

In [6]:
answers_list = chain.batch(inputs)

In [7]:
for ans in answers_list:
    print(ans)
    print("------------------------------------------------")

To determine Harold's relationship to Marie, let's first outline the family relationships based on the information provided:

1. (Bernardo, has father, Antonio) → Antonio is Bernardo's father.
2. (Antonio, has son, Bernardo) → Bernardo is Antonio's son.
3. (Antonio, has father, Harold) → Harold is Antonio's father.
4. (Harold, has son, Antonio) → Antonio is Harold's son.
5. (Antonio, has mother, Shantel) → Shantel is Antonio's mother.
6. (Shantel, has son, Antonio) → Antonio is Shantel’s son.
7. (Shantel, has daughter, Marie) → Marie is Shantel’s daughter.
8. (Marie, has mother, Shantel) → Shantel is Marie’s mother.

From these relationships, we can conclude the following:

- Harold is Antonio's father (from 3 and 4).
- Shantel is both Antonio's mother and Marie's mother (from 5, 6, 7, and 8). Therefore, Antonio and Marie are siblings.

Since Harold is Antonio’s father and Marie is Antonio’s sister, this makes Harold Marie’s father.

Therefore, Harold is Marie's father.
---------------

In [ ]:
# answer for 1 :- father (correct)
# answer for 2 :- mother (correct)
# answer for 3 :- daughter (correct)
# answer for 4 :- niece (correct)

## 2. Using Falcor

In [8]:
from collections import defaultdict
import re
import json

# Initialize a graph
falcor_graph = {
    "persons": defaultdict(dict)
}

In [9]:
def parse_relationship_string(relationship_str):
    # Extract all (subject, predicate, object) triples using regex
    triple_pattern = r"\(([^,]+),\s*has ([^,]+),\s*([^)]+)\)"
    matches = re.findall(triple_pattern, relationship_str)
    return matches
    

def build_falcor_graph(triples):
    graph = {"persons": defaultdict(lambda: defaultdict(list))}

    for subj, relation, obj in triples:
        subj = subj.strip()
        obj = obj.strip()
        relation = relation.strip().lower() + "s"  # plural (e.g., father → fathers)
        
        # Add the relationship to the graph
        graph["persons"][subj][relation].append({"$ref": ["persons", obj]})
    
    return graph
    
i = 0
for input_itm in inputs:
    input_itm_rel = parse_relationship_string(input_itm["relationships"])
    falcor_graph = build_falcor_graph(input_itm_rel)
    inputs[i]["relationships"] = json.dumps(falcor_graph)
    i += 1

inputs

[{'relationships': '{"persons": {"Bernardo": {"fathers": [{"$ref": ["persons", "Antonio"]}]}, "Antonio": {"sons": [{"$ref": ["persons", "Bernardo"]}], "fathers": [{"$ref": ["persons", "Harold"]}], "mothers": [{"$ref": ["persons", "Shantel"]}]}, "Harold": {"sons": [{"$ref": ["persons", "Antonio"]}]}, "Shantel": {"sons": [{"$ref": ["persons", "Antonio"]}], "daughters": [{"$ref": ["persons", "Marie"]}]}, "Marie": {"mothers": [{"$ref": ["persons", "Shantel"]}]}}}',
  'question': "Harold is Marie's ____"},
 {'relationships': '{"persons": {"Robert": {"brothers": [{"$ref": ["persons", "Antonio"]}], "daughters": [{"$ref": ["persons", "Michelle"]}]}, "Marie": {"daughters": [{"$ref": ["persons", "Michelle"]}], "babys": [{"$ref": ["persons", "Baby"]}], "mothers": [{"$ref": ["persons", "Shantel"]}]}, "Shantel": {"granddaughters": [{"$ref": ["persons", "Baby"]}]}}}',
  'question': "Shantel is Antonio's ____"},
 {'relationships': '{"persons": {"Samuel": {"grandfathers": [{"$ref": ["persons", "Harold

In [10]:
prompt_str = (
    f"You are an expert in understanding family relationships."
    f"You are given a list of relationships between individuals in Falcor format."
    f"Your task is to figure out the relationship :- {{question}}"
    f""
    f"Relationships in Falcor format :"
    f"{{relationships}}"
)

prompt = ChatPromptTemplate.from_template(prompt_str)

In [11]:
chain = prompt | model | StrOutputParser()

In [12]:
answers_list = chain.batch(inputs)

In [13]:
for ans in answers_list:
    print(ans)
    print("------------------------------------------------")

To determine Harold's relationship with Marie, let's break down the given information:

1. Harold has a son, Antonio.
2. Antonio is the father of Bernardo and the son of Harold and Shantel.
3. Shantel has a daughter, Marie.
4. Marie's mother is Shantel.

Now, combining these pieces of information:

- Harold and Shantel are parents to Antonio.
- Shantel is also a parent to Marie, so Antonio and Marie are siblings (sharing Shantel as a common parent).

This makes Harold the father of Antonio and the stepfather to Marie (since he is not listed as her biological father), but since we only use the biological relationship in most cases when assessing these terms and based on the given information explicitly linking him as a parent to Antonio but not to Marie, the relationship of Harold to Marie through Shantel and Antonio would commonly be understood as "father-in-law" in the broader sense of the family structure.

Thus, Harold is Marie's ***stepfather***.
-----------------------------------

In [ ]:
# answer for 1 :- father (correct)
# answer for 2 :- mother (correct)
# answer for 3 :- daughter (incorrect)
# answer for 4 :- niece (correct)